### Assignment 1: Missionary and Cannibals 



In [47]:
from abc import ABCMeta, abstractmethod
import queue as Q
import numpy as np
# Basic data structure Stack
#
#
class Stack:
     def __init__(self):
         self.items = []
     def isEmpty(self):
         return self.items == []
     def push(self, item):
         self.items.append(item)
     def pop(self):
         return self.items.pop()
     def peek(self):
         return self.items[len(self.items)-1]
     def size(self):
         return len(self.items)
#
# Basic data structure Queue
# (Use my own Queue instead of import) 
#        
class Queue:
    def __init__(self):
        self.items = []
    def isEmpty(self):
        return self.items == []
    def add(self, item):
        self.items.insert(0,item)
    def get(self):
        return self.items.pop()
    def size(self):
        return len(self.items)
    
class Solver(metaclass=ABCMeta):
    """
     ABC=Abstract base class
     Abstract Solver class that searches through the state space, called by the Board class
    """
    @abstractmethod
    def get(self):
         raise NotImplementedError()
    @abstractmethod
    def add(self):
         raise NotImplementedError()
     
class Solver1(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     uses a Queue
    """
    def __init__(self):
        self.queue=Queue()
    def get(self):
        return self.queue.get()
    def add(self,board):
        self.queue.add(board)

class Solver2(Solver):
    """
     Concrete solver: must implement abstract methods from abstract class Solver
     Uses a Stack
    """
    def __init__(self):
        self.stack=Stack()
    def get(self):
        return self.stack.pop()
    def add(self,board):
        self.stack.push(board)

class Solver3:
    def __init__(self, n, c):
        self.pq = Q.PriorityQueue()
        self.goal = np.array([0, 0, n, n, 0])

    def get(self):
        if not self.pq.empty():
            priority, board = self.pq.get()
            return np.array(board), priority
        return None, None

    def add(self, board):
        priority = self.heuristic(board)
        self.pq.put((priority, tuple(board)))

    def heuristic(self, board):
        return board[0] + board[1]  # M_right + C_right

Copy also the Class Board but you need edit them extensively. 

For Board class:
1. Remove all the class variables
2. Edit the __init__ function, remove the initialize board code
3. Edit printBoard for the state (board) representation you used
4. Delete these functions: getBlankPosition, getRandomMove, scramble
5. Replace makeMove, isValidMove and getMoves with code you created here.
6. In the solve function, you may need to write your own code to replace this if you used a numpy array: 
        if not (temp in self.used):
   This code is just used to check that the previous move have not been used before. Old moves are stored in self.used                 

    Here is a sample code to check if the current board position have been used before.
    
        def isNotUsed(self, used, board):
            for u in used:
                #print(u,board)
                if np.array_equal(u,board) :
                    return False
            return True

There are some minor changes here and there that you should be able to fix.

In [59]:
class Board:
    def __init__(self, n, c):
        self.n = n
        self.c = c
        self.board = np.array([n, n, 0, 0, 1])  # [M_right, C_right, M_left, C_left, boat_position]
        self.goal = np.array([0, 0, n, n, 0])
        self.used = set()
        self.solver = Solver3(n, c)
        print(f"Start with n = {n} and c = {c}")

    def isValidBoard(self, board):
        M_right, C_right, M_left, C_left, _ = board
        if M_right < 0 or C_right < 0 or M_left < 0 or C_left < 0:
            return False
        if (M_right < C_right and M_right > 0) or (M_left < C_left and M_left > 0):
            return False
        return True

    def makeMove(self, board, move):
        new_board = board.copy()
        m, c = move
        if board[4] == 1:  # boat on right
            new_board[:4] += [-m, -c, m, c]
            new_board[4] = 0
        else:  # boat on left
            new_board[:4] += [m, c, -m, -c]
            new_board[4] = 1
        return new_board

    def getMoves(self, board):
        moves = []
        boat_side = board[4]
        m_on_side = board[0] if boat_side == 1 else board[2]
        c_on_side = board[1] if boat_side == 1 else board[3]

        for m in range(min(m_on_side, self.c) + 1):
            for c in range(min(c_on_side, self.c - m) + 1):
                if 0 < m + c <= self.c:
                    move = (m, c) if boat_side == 1 else (-m, -c)
                    new_board = self.makeMove(board, move)
                    if self.isValidBoard(new_board):
                        moves.append(move)
        return moves

    def solve(self):
        self.used = set()
        self.solver.add(self.board)
        steps = 0
        max_steps = 100000  # Increased to handle more complex cases

        while steps < max_steps:
            board, _ = self.solver.get()
            if board is None:
                print("No solution found.")
                return False
            
            board_tuple = tuple(board)
            if board_tuple in self.used:
                continue
            self.used.add(board_tuple)
            
            print(f"\nStep {steps + 1}:")
            self.printBoard(board)
            
            if np.array_equal(board, self.goal):
                print("Solution found!")
                return True
            
            moves = self.getMoves(board)
            print(f"Valid Moves from Current State: {moves}")
            
            for move in moves:
                new_board = self.makeMove(board, move)
                if tuple(new_board) not in self.used:
                    self.solver.add(new_board)
            
            steps += 1

        print("No solution found within the step limit.")
        return False

    def printBoard(self, board):
        M_right, C_right, M_left, C_left, boat = board
        print(f"Right Bank - M: {M_right}, C: {C_right} | Left Bank - M: {M_left}, C: {C_left} | Boat: {'Right' if boat == 1 else 'Left'}")

### Run and Test

If you have correctly edited the code, then it should run.

1. Draw the search tree from the program result (insert into this notebook).
2. Give the solution to each problem


In [61]:
def main():
    game=Board(3,2)    
    # game=Board(4,3)    
    # game=Board(5,3)        
    game.solve()

if __name__ == '__main__':
    main()

Start with n = 3 and c = 2

Step 1:
Right Bank - M: 3, C: 3 | Left Bank - M: 0, C: 0 | Boat: Right
Valid Moves from Current State: [(0, 1), (0, 2), (1, 1)]

Step 2:
Right Bank - M: 2, C: 2 | Left Bank - M: 1, C: 1 | Boat: Left
Valid Moves from Current State: [(-1, -1)]

Step 3:
Right Bank - M: 1, C: 1 | Left Bank - M: 2, C: 2 | Boat: Right
Valid Moves from Current State: [(1, 0), (1, 1)]

Step 4:
Right Bank - M: 0, C: 0 | Left Bank - M: 3, C: 3 | Boat: Left
Solution found!
